In [1]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.llms import KoboldApiLLM

In [2]:
llm = KoboldApiLLM(endpoint="http://localhost:5001")

In [3]:
from langchain.tools import StructuredTool

In [4]:
SUPERVISOR_API = "http://localhost:3000"

In [5]:
import requests

token_response = requests.post(
            f'{SUPERVISOR_API}/token', data={'username': "a", 'password': "a"}, timeout=600)

token_text = token_response.json().get('access_token')

In [6]:
token_instance = token_text

In [7]:
def code_interpreter_lite(code: str) -> str:
            """Execute the python code and return the result."""
            # handle markdown
            def extract_code_from_markdown(md_text):
                # Using regex to extract text between ```
                pattern = r"```[\w]*\n(.*?)```"
                match = re.search(pattern, md_text, re.DOTALL)
                if match:
                    return match.group(1).strip()
                else:
                    # might not be markdown
                    return md_text
            code = extract_code_from_markdown(code)

            code_response = requests.post(
                f'{SUPERVISOR_API}/run', headers={
                    'Authorization': f'Bearer {token_instance}'
                }, json={
                    'code_string': code
                }, timeout=600)
            if code_response.status_code != 200:
                raise gr.Error("No container created yet")
                # return {
                #     chatbot: chatbot_instance,
                #     msg: msg_instance,
                # }
            result = code_response.json()

            return result
        

In [8]:
tool = StructuredTool.from_function(
    func=code_interpreter_lite, name="CIL", description="useful for running python code. The input should be a string of python code.")

In [9]:
agent = initialize_agent(
    [tool], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [10]:
agent.run("What is 5 raised to the 0.43 power?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: 5 ** 0.43 ** 1 = 2.786932244205051e+02

# In[ ]:
print("The answer is:", round(2.786932244205051e+02, 2)) # prints "The answer is